In [1]:
import os

def get_machine_storage():
    result=os.statvfs('/')
    block_size=result.f_frsize
    total_blocks=result.f_blocks
    free_blocks=result.f_bfree
    # giga=1024*1024*1024
    giga=1000*1000*1000
    total_size=total_blocks*block_size/giga
    free_size=free_blocks*block_size/giga
    print('total_size = %s' % total_size)
    print('free_size = %s' % free_size)

get_machine_storage()

total_size = 316.9979392
free_size = 90.543136768


In [2]:
import os

def get_size(start_path = '.'):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    return total_size

print(get_size('')/1024/1024/1024, 'Gbytes')

0.0 Gbytes


In [3]:
import eco2ai
tracker = eco2ai.Tracker(project_name="lct_12_sber_multimodal_recsys", 
                         experiment_description="naive_v1_trx_v1_2_dial_geo_v1_2_new_fea_v4_iter750", ignore_warnings=True)
tracker.start()

In [4]:
import polars as pol
import pandas as pd
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold
import catboost as cat
import datetime
import pickle
import sklearn

import multiprocessing
print('cpu', multiprocessing.cpu_count())

import psutil    
print('ram', round(psutil.virtual_memory().total/1024/1024/1024,1), 'gb')  # total physical memory in Bytes

cpu 64
ram 629.9 gb


In [5]:
print('catboost', cat.__version__)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('polars', pol.__version__)
print('sklearn', sklearn.__version__)

catboost 1.2.2
pandas 1.4.3
numpy 1.26.0
polars 0.20.17
sklearn 1.3.1


## 1. Подготовка фич из файлов target_train и test_target_b

 - сколько раз в предыдущих месяцах встречался каждый из таргетов или любой их них, как давно они встречались
 - в каком из месяцев последний раз встречался каждый из таргетов или люой из них и сколько месяцев прошло с тех пор

In [6]:
%%time
target_train = pol.read_parquet("Hackathon/train_target.parquet/*")
test_target_b = pol.read_parquet("Hackathon/test_target_b.parquet/*")
print(target_train.shape, test_target_b.shape)

(10246704, 6) (1439362, 6)
CPU times: user 1.75 s, sys: 1.88 s, total: 3.64 s
Wall time: 618 ms


In [7]:
mon_dict = {j:i+2 for i, j in enumerate(target_train['mon'].value_counts().sort('mon')['mon'].to_numpy())}
mon_dict

{'2022-02-28': 2,
 '2022-03-31': 3,
 '2022-04-30': 4,
 '2022-05-31': 5,
 '2022-06-30': 6,
 '2022-07-31': 7,
 '2022-08-31': 8,
 '2022-09-30': 9,
 '2022-10-31': 10,
 '2022-11-30': 11,
 '2022-12-31': 12,
 '2023-01-31': 13}

In [8]:
target_train = target_train.with_columns(pol.col('mon').replace(mon_dict).alias('new_mon').cast(int))
target_train.head(2)

mon,target_1,target_2,target_3,target_4,client_id,new_mon
str,i32,i32,i32,i32,str,i64
"""2022-06-30""",0,0,0,0,"""1d4ebf30ab5b98…",6
"""2022-07-31""",0,0,0,0,"""1d4ebf30ab5b98…",7


In [9]:
test_target_b = test_target_b.with_columns(pol.col('mon').replace(mon_dict).alias('new_mon').cast(int))
test_target_b.head(2)

mon,target_1,target_2,target_3,target_4,client_id,new_mon
str,i32,i32,i32,i32,str,i64
"""2022-05-31""",0,0,0,0,"""2b7ff0c1c99cef…",5
"""2022-05-31""",0,0,0,0,"""0433d23e224b7a…",5


In [10]:
print(target_train.shape, test_target_b.shape)
print(target_train.unique().shape, test_target_b.unique().shape)

(10246704, 7) (1439362, 7)
(10246704, 7) (1407671, 7)


In [11]:
test_target_b.filter(pol.lit(test_target_b.is_duplicated())).sum()

mon,target_1,target_2,target_3,target_4,client_id,new_mon
str,i32,i32,i32,i32,str,i64
null,0,0,0,0,null,396792


In [12]:
#удаляем дубликаты из test_target_b
test_target_b = test_target_b.unique()
print(test_target_b.shape)

(1407671, 7)


In [13]:
# hold-out валидации - 30% от target'ов за 3 последних месяца
target_valid = target_train.filter(pol.col('new_mon')>10).sample(fraction=0.3, seed=37)
target_valid = target_valid.with_columns(pol.lit(1).alias('valid_flag'))
print(target_valid.shape, target_valid['client_id'].n_unique())

(768502, 8) 560567


In [14]:
target_valid['new_mon'].value_counts()

new_mon,count
i64,u32
12,256255
11,255900
13,256347


In [15]:
# объединяем train и test, чтобы историческая информация о поведении тестовых пользователей тоже использовалась для обучения
target_train = pol.concat([target_train, test_target_b])
print(target_train.shape, target_train['client_id'].n_unique())

(11654375, 7) 994380


In [16]:
target_train['target_1'].value_counts()

target_1,count
i32,u32
0,11552098
1,102277


In [17]:
res = []
for m in range(3,14):
    print(m, end=' ')
    a = target_train.filter(pol.col('new_mon')<m)
    a = a.with_columns(target=pol.sum_horizontal(['target_'+str(i) for i in range(1,5)]))
    r = a.group_by('client_id').agg([pol.col('target_1').sum(), pol.col('target_2').sum(), pol.col('target_3').sum(), pol.col('target_4').sum(), pol.col('target').sum()])
    r = r.rename({'target_'+str(i):'target_'+str(i)+'_cumsum' for i in range(1,5)})
    r = r.rename({'target':'target_cumsum'})
    r = r.with_columns(pol.lit(m).alias('new_mon'))
    
    for n in range(1,5):
        t = a.with_columns((pol.col('target_'+str(n)).cast(bool)*pol.col('new_mon')).alias('last_target_'+str(n)+'_month')).group_by('client_id').agg(pol.col('last_target_'+str(n)+'_month').max())
        t = t.with_columns(((pol.lit(m)-pol.col('last_target_'+str(n)+'_month'))*pol.col('last_target_'+str(n)+'_month')).alias('delta_last_target_'+str(n)+'_month'))
        r = r.join(t, on='client_id', how='left')
        del t

    t = a.with_columns((pol.col('target').cast(bool)*pol.col('new_mon')).alias('last_target_month')).group_by('client_id').agg(pol.col('last_target_month').max())
    t = t.with_columns(((pol.lit(m)-pol.col('last_target_month'))*pol.col('last_target_month')).alias('delta_last_target_month'))
    r = r.join(t, on='client_id', how='left')
    del t
    
    res.append(r)
    del r, a

res = pol.concat(res)
print(res.shape)
res.head(2)

3 4 5 6 7 8 9 10 11 12 13 (10938180, 17)


client_id,target_1_cumsum,target_2_cumsum,target_3_cumsum,target_4_cumsum,target_cumsum,new_mon,last_target_1_month,delta_last_target_1_month,last_target_2_month,delta_last_target_2_month,last_target_3_month,delta_last_target_3_month,last_target_4_month,delta_last_target_4_month,last_target_month,delta_last_target_month
str,i32,i32,i32,i32,i32,i32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""1c10b1cea942b9…",0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0
"""2d584877943555…",0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0


In [18]:
sub = test_target_b.group_by('client_id').agg(pol.col('new_mon').max()).with_columns(pol.col('new_mon')+pol.lit(1))
print(sub.shape)
sub.head(2)

(140488, 2)


client_id,new_mon
str,i64
"""9573ebb8df1a73…",13
"""2ab37055966d13…",13


In [19]:
sub['new_mon'].value_counts().sort('new_mon')

new_mon,count
i64,u32
11,46086
12,45525
13,48877


In [20]:
res_test = []
for m in range(11,14):
    print(m, end=' ')
    a = test_target_b.filter(pol.col('new_mon')<m)
    a = a.with_columns(target=pol.sum_horizontal(['target_'+str(i) for i in range(1,5)]))
    r = a.group_by('client_id').agg([pol.col('target_1').sum(), pol.col('target_2').sum(), pol.col('target_3').sum(), pol.col('target_4').sum(), pol.col('target').sum()])
    r = r.rename({'target_'+str(i):'target_'+str(i)+'_cumsum' for i in range(1,5)})
    r = r.rename({'target':'target_cumsum'})
    r = r.with_columns(pol.lit(m).alias('new_mon'))
    
    for n in range(1,5):
        t = a.with_columns((pol.col('target_'+str(n)).cast(bool)*pol.col('new_mon')).alias('last_target_'+str(n)+'_month')).group_by('client_id').agg(pol.col('last_target_'+str(n)+'_month').max())
        t = t.with_columns(((pol.lit(m)-pol.col('last_target_'+str(n)+'_month'))*pol.col('last_target_'+str(n)+'_month')).alias('delta_last_target_'+str(n)+'_month'))
        r = r.join(t, on='client_id', how='left')
        del t

    t = a.with_columns((pol.col('target').cast(bool)*pol.col('new_mon')).alias('last_target_month')).group_by('client_id').agg(pol.col('last_target_month').max())
    t = t.with_columns(((pol.lit(m)-pol.col('last_target_month'))*pol.col('last_target_month')).alias('delta_last_target_month'))
    r = r.join(t, on='client_id', how='left')
    del t
    
    res_test.append(r)
    del r, a

res_test = pol.concat(res_test)
print(res_test.shape)
res_test.head(2)

11 12 13 (421464, 17)


client_id,target_1_cumsum,target_2_cumsum,target_3_cumsum,target_4_cumsum,target_cumsum,new_mon,last_target_1_month,delta_last_target_1_month,last_target_2_month,delta_last_target_2_month,last_target_3_month,delta_last_target_3_month,last_target_4_month,delta_last_target_4_month,last_target_month,delta_last_target_month
str,i32,i32,i32,i32,i32,i32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""6f26ff956ae03f…",0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0
"""a8be404236eff7…",0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0


In [21]:
print(res.shape)
res = res.join(target_train.with_columns(pol.col('new_mon').cast(pol.datatypes.Int32)), on=['client_id','new_mon'], how='left')
print(res.shape)
res.head(2)

(10938180, 17)
(10938180, 22)


client_id,target_1_cumsum,target_2_cumsum,target_3_cumsum,target_4_cumsum,target_cumsum,new_mon,last_target_1_month,delta_last_target_1_month,last_target_2_month,delta_last_target_2_month,last_target_3_month,delta_last_target_3_month,last_target_4_month,delta_last_target_4_month,last_target_month,delta_last_target_month,mon,target_1,target_2,target_3,target_4
str,i32,i32,i32,i32,i32,i32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i32,i32,i32,i32
"""1c10b1cea942b9…",0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,"""2022-03-31""",0,0,0,0
"""2d584877943555…",0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,"""2022-03-31""",0,0,0,0


In [22]:
print(res_test.shape)
res_test = res_test.join(sub.with_columns(pol.col('new_mon').cast(pol.datatypes.Int32)), on=['client_id','new_mon'], how='inner')
print(res_test.shape)
res_test.head(2)

(421464, 17)
(140488, 17)


client_id,target_1_cumsum,target_2_cumsum,target_3_cumsum,target_4_cumsum,target_cumsum,new_mon,last_target_1_month,delta_last_target_1_month,last_target_2_month,delta_last_target_2_month,last_target_3_month,delta_last_target_3_month,last_target_4_month,delta_last_target_4_month,last_target_month,delta_last_target_month
str,i32,i32,i32,i32,i32,i32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""6f26ff956ae03f…",0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0
"""d4c2b224c3f522…",0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0


In [23]:
# присоединяем таргет и отделяем hold-out валидацию

res = res.sort(('client_id', 'new_mon'))
print(res.shape)

res = res.join(target_valid.with_columns(pol.col('new_mon').cast(pol.datatypes.Int32)).select(['client_id', 'new_mon', 'valid_flag']), 
               on=['client_id','new_mon'], how='left')
res = res.with_columns(pol.col('valid_flag').fill_null(0))

res_valid = res.filter(pol.col('valid_flag')==1)
res = res.filter(pol.col('valid_flag')==0)
print(res.shape, res_valid.shape)

(10938180, 22)
(10169678, 23) (768502, 23)


## 2. Transactions' embeddings

 - присоединяем эмбеддинги по тразакциям, посчитанные в ноутбуке trx_pyspark_v1_2_train_test

In [24]:
train_trx = pol.concat([pol.read_parquet("train_pyspark_v1_2_m_1_2.parquet"), pol.read_parquet("train_pyspark_v1_2_m_3_4.parquet"),
                       pol.read_parquet("train_pyspark_v1_2_m_5_6.parquet"), pol.read_parquet("train_pyspark_v1_2_m_7_8.parquet"),
                       pol.read_parquet("train_pyspark_v1_2_m_9_10.parquet"), pol.read_parquet("train_pyspark_v1_2_m_11_12.parquet")])

not_only_trx = pol.read_parquet("test_pyspark_v1_2.parquet")

print(train_trx.shape, not_only_trx.shape)

Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:28:04 UTC)" was missed by 0:00:01.490454


(12976452, 257) (140488, 257)


In [25]:
train_trx.head(2)

client_id,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,…,emb_0219,emb_0220,emb_0221,emb_0222,emb_0223,emb_0224,emb_0225,emb_0226,emb_0227,emb_0228,emb_0229,emb_0230,emb_0231,emb_0232,emb_0233,emb_0234,emb_0235,emb_0236,emb_0237,emb_0238,emb_0239,emb_0240,emb_0241,emb_0242,emb_0243,emb_0244,emb_0245,emb_0246,emb_0247,emb_0248,emb_0249,emb_0250,emb_0251,emb_0252,emb_0253,emb_0254,emb_0255
str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""0002cdf0478690…",0.951077,0.999686,1.0,1.0,-0.880187,-0.284764,0.932937,-0.999406,-0.379062,1.0,-0.999998,-1.0,-0.845215,0.356624,0.991651,-0.770026,-0.999574,1.0,0.385103,0.999935,-0.976532,0.172674,0.910173,-1.0,0.991045,-0.377919,-1.0,-0.032141,-1.0,-0.999649,-1.0,0.913607,1.0,0.998854,0.999994,-0.29577,…,-0.925463,-0.859127,0.975712,1.0,0.999963,-0.765673,0.999999,-0.999998,0.976849,0.470971,1.0,0.677713,-1.0,-0.995411,-0.999993,1.0,-0.973607,0.989827,-0.963541,-0.999931,0.720909,-1.0,-0.99996,0.999987,-0.989661,-0.130845,-0.999146,0.999903,1.0,-0.744656,-0.995929,-1.0,0.997417,0.864674,0.999778,-0.999984,-0.999224
"""0002cdf0478690…",0.948332,0.999748,1.0,1.0,-0.8721,-0.300032,0.935208,-0.999379,-0.375694,1.0,-0.999999,-1.0,-0.833496,0.355374,0.99847,-0.761078,-0.999583,1.0,0.406482,0.999932,-0.97388,0.205471,0.916694,-1.0,0.991321,-0.382264,-1.0,-0.029452,-1.0,-0.999889,-1.0,0.918849,1.0,0.998949,0.999998,-0.325161,…,-0.926869,-0.854659,0.978257,1.0,0.999974,-0.754931,0.999999,-0.999999,0.995525,0.473218,1.0,0.681318,-1.0,-0.996015,-0.999995,1.0,-0.974455,0.946113,-0.963092,-0.999949,0.725508,-1.0,-0.999955,0.999997,-0.997573,-0.160712,-0.999691,0.999909,1.0,-0.7255,-0.995878,-1.0,0.997312,0.86811,0.999863,-0.999986,-0.999353


In [26]:
train_trx.head(1).to_pandas().loc[0,'client_id']

'0002cdf04786903813a2f4f123e4ad2ce04c586760472a81e68460883bb9f369_month=1'

In [27]:
%%time
train_trx = train_trx.with_columns(pol.col('client_id').map_elements(lambda x: int(x.split('=')[-1])+1).alias('new_mon'))
train_trx = train_trx.with_columns(pol.col('client_id').map_elements(lambda x: x.split('_')[0]).alias('client_id'))
train_trx.head(2)

<timed exec>:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
<timed exec>:2: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


CPU times: user 9.81 s, sys: 3.76 s, total: 13.6 s
Wall time: 13.6 s


client_id,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,…,emb_0220,emb_0221,emb_0222,emb_0223,emb_0224,emb_0225,emb_0226,emb_0227,emb_0228,emb_0229,emb_0230,emb_0231,emb_0232,emb_0233,emb_0234,emb_0235,emb_0236,emb_0237,emb_0238,emb_0239,emb_0240,emb_0241,emb_0242,emb_0243,emb_0244,emb_0245,emb_0246,emb_0247,emb_0248,emb_0249,emb_0250,emb_0251,emb_0252,emb_0253,emb_0254,emb_0255,new_mon
str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i64
"""0002cdf0478690…",0.951077,0.999686,1.0,1.0,-0.880187,-0.284764,0.932937,-0.999406,-0.379062,1.0,-0.999998,-1.0,-0.845215,0.356624,0.991651,-0.770026,-0.999574,1.0,0.385103,0.999935,-0.976532,0.172674,0.910173,-1.0,0.991045,-0.377919,-1.0,-0.032141,-1.0,-0.999649,-1.0,0.913607,1.0,0.998854,0.999994,-0.29577,…,-0.859127,0.975712,1.0,0.999963,-0.765673,0.999999,-0.999998,0.976849,0.470971,1.0,0.677713,-1.0,-0.995411,-0.999993,1.0,-0.973607,0.989827,-0.963541,-0.999931,0.720909,-1.0,-0.99996,0.999987,-0.989661,-0.130845,-0.999146,0.999903,1.0,-0.744656,-0.995929,-1.0,0.997417,0.864674,0.999778,-0.999984,-0.999224,2
"""0002cdf0478690…",0.948332,0.999748,1.0,1.0,-0.8721,-0.300032,0.935208,-0.999379,-0.375694,1.0,-0.999999,-1.0,-0.833496,0.355374,0.99847,-0.761078,-0.999583,1.0,0.406482,0.999932,-0.97388,0.205471,0.916694,-1.0,0.991321,-0.382264,-1.0,-0.029452,-1.0,-0.999889,-1.0,0.918849,1.0,0.998949,0.999998,-0.325161,…,-0.854659,0.978257,1.0,0.999974,-0.754931,0.999999,-0.999999,0.995525,0.473218,1.0,0.681318,-1.0,-0.996015,-0.999995,1.0,-0.974455,0.946113,-0.963092,-0.999949,0.725508,-1.0,-0.999955,0.999997,-0.997573,-0.160712,-0.999691,0.999909,1.0,-0.7255,-0.995878,-1.0,0.997312,0.86811,0.999863,-0.999986,-0.999353,3


In [28]:
%%time
res = res.join(train_trx.with_columns(pol.col('new_mon').cast(pol.datatypes.Int32)), on=['client_id','new_mon'], how='left').fill_null(0)
res_valid = res_valid.join(train_trx.with_columns(pol.col('new_mon').cast(pol.datatypes.Int32)), on=['client_id','new_mon'], how='left').fill_null(0)
res_test = res_test.join(not_only_trx, on=['client_id'], how='left').fill_null(0)

print(res.shape, res_valid.shape, res_test.shape)

(10169678, 279) (768502, 279) (140488, 273)
CPU times: user 1min 2s, sys: 57.6 s, total: 1min 59s
Wall time: 3.26 s


In [29]:
del train_trx, not_only_trx

In [30]:
#res_p = res.to_pandas()
#res_valid_p = res_valid.to_pandas()
#res_test_p = res_test.to_pandas()

## 3. Dial embeddings

 - присоединяем усреднённые эмбеддинги по звонкам (в среднем на клиента около 4 диалогов, так что проще усреднить, чем обрабатывать как последовательность)
 - табличные фичи: сколько раз клиент звонил всего и за последний месяц, сколько времени прошло с последнего и среднего звонка

In [31]:
%%time
dial_train = pol.read_parquet("Hackathon/dial_train.parquet/*")
print(dial_train.shape, dial_train['client_id'].n_unique())
dial_train.head()

(1215209, 3) 326803
CPU times: user 23.8 s, sys: 11.7 s, total: 35.5 s
Wall time: 28.9 s


client_id,event_time,embedding
str,datetime[μs],list[f32]
"""a039ad3b595d4f…",2022-05-02 12:24:30.565231,"[-0.003248, 0.140231, … -0.010614]"
"""a039ad3b595d4f…",2022-10-28 09:05:18.394466,"[0.058927, -0.007723, … 0.086532]"
"""a039ad3b595d4f…",2022-12-15 08:36:26.987451,"[0.079807, -0.003912, … 0.095575]"
"""a060e69e9e049a…",2022-09-13 13:37:03.710475,"[0.523752, -0.30542, … 0.568542]"
"""a08c690dd972d2…",2022-05-02 09:12:22.795170,"[-0.009235, -0.069714, … -0.062696]"


In [32]:
%%time
dial_test = pol.read_parquet("Hackathon/dial_test.parquet/*")
print(dial_test.shape, dial_test['client_id'].n_unique())
dial_test.head()

(286526, 3) 81425
CPU times: user 7.36 s, sys: 10.3 s, total: 17.7 s
Wall time: 9.2 s


client_id,event_time,embedding
str,datetime[ns],list[f32]
"""08b3569cdfd015…",2022-02-26 10:53:11.777539,"[0.110589, -0.000545, … 0.134537]"
"""08b3569cdfd015…",2022-05-01 09:37:13.140523,"[0.00209, 0.072185, … 0.015157]"
"""08b3569cdfd015…",2022-05-09 08:48:24.248675,"[0.194512, -0.032053, … 0.160809]"
"""08b3569cdfd015…",2022-03-21 07:02:23.951399,"[0.045035, 0.042004, … -0.112547]"
"""08b3569cdfd015…",2022-03-19 08:55:18.098455,"[0.330715, -0.023786, … 0.342075]"


In [33]:
dial_train.head(1).to_pandas().loc[0,'embedding'].shape

(768,)

In [34]:
print(dial_train.shape, dial_test.shape)
print(dial_train.unique().shape, dial_test.unique().shape)

(1215209, 3) (286526, 3)
(1215209, 3) (283658, 3)


In [35]:
dial_test = dial_test.unique()
print(dial_test.shape)

(283658, 3)


In [36]:
dial_train = pol.concat([dial_train.with_columns(pol.col('event_time').dt.cast_time_unit("ns")), dial_test])
print(dial_train.shape, dial_train['client_id'].n_unique())
dial_train.head(2)

Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:29:04 UTC)" was missed by 0:00:01.931788


(1498867, 3) 408228


client_id,event_time,embedding
str,datetime[ns],list[f32]
"""a039ad3b595d4f…",2022-05-02 12:24:30.565231,"[-0.003248, 0.140231, … -0.010614]"
"""a039ad3b595d4f…",2022-10-28 09:05:18.394466,"[0.058927, -0.007723, … 0.086532]"


In [37]:
%%time
dial_res = []
for m in range(2,14):
    t = dial_train.filter(pol.col('event_time')<datetime.datetime(2022+(m-1)//12, m%12+int((m%12)==0)*12, 1, 0, 0, 0, 0))
    print(m, t.shape, t['client_id'].n_unique())    
    t_embs = pol.from_pandas(t.to_pandas().groupby('client_id')['embedding'].mean().reset_index(drop=False))
    t_embs = t_embs.with_columns(pol.col("embedding").list.to_struct()).unnest("embedding")
    t_embs = t_embs.with_columns(pol.lit(m).alias('new_mon'))

    a = t.group_by('client_id').agg(pol.col('event_time').count().alias('num_dials'))
    t_embs = t_embs.join(a, on='client_id', how='left')
    
    a = t.group_by('client_id').agg(pol.col('event_time').max().alias('seconds_from_last_call'))
    a = a.with_columns((datetime.datetime(2022+(m-1)//12, m%12+int((m%12)==0)*12, 1, 0, 0, 0, 0)-pol.col('seconds_from_last_call')).dt.total_seconds().cast(int).alias('seconds_from_last_call'))
    t_embs = t_embs.join(a, on='client_id', how='left')    
    
    a = t.group_by('client_id').agg(pol.col('event_time').mean().alias('seconds_from_mean_call'))
    a = a.with_columns((datetime.datetime(2022+(m-1)//12, m%12+int((m%12)==0)*12, 1, 0, 0, 0, 0)-pol.col('seconds_from_mean_call')).dt.total_seconds().cast(int).alias('seconds_from_mean_call'))
    t_embs = t_embs.join(a, on='client_id', how='left')   

    t = dial_train.filter(pol.col('event_time')<datetime.datetime(2022+(m-1)//12, m%12+int((m%12)==0)*12, 1, 0, 0, 0, 0))
    t = t.filter(pol.col('event_time')>=datetime.datetime(2022+(m-2)//12, (m-1)%12+int(((m-1)%12)==0)*12, 1, 0, 0, 0, 0))

    a = t.group_by('client_id').agg(pol.col('event_time').count().alias('num_dials_lm')).fill_null(0)
    t_embs = t_embs.join(a, on='client_id', how='left')

    #t_embs1 = pol.from_pandas(t.to_pandas().groupby('client_id')['embedding'].mean().reset_index(drop=False))
    #t_embs1 = t_embs1.with_columns(pol.col("embedding").list.to_struct()).unnest("embedding")
    #t_embs1 = t_embs1.rename({'field_'+str(i):'lm_field_'+str(i) for i in range(768)})
    #t_embs = t_embs.join(t_embs1, on='client_id', how='left')
    
    dial_res.append(t_embs)
    del t, t_embs, a#, t_embs1

dial_res = pol.concat(dial_res)
print(dial_res.shape)
dial_res.head(2)

2 (63523, 3) 44639
3 (110439, 3) 69426
4 (177147, 3) 100839
5 (244003, 3) 128081
6 (321556, 3) 160539
7 (484030, 3) 212480
8 (679600, 3) 261285
9 (866150, 3) 303380
10 (1039258, 3) 337300
11 (1210599, 3) 367663
12 (1365307, 3) 391722


Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:34:44 UTC)" was missed by 0:00:02.553646


13 (1498867, 3) 408228


Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:35:44 UTC)" was missed by 0:00:07.061439


(2785582, 774)
CPU times: user 6min 33s, sys: 2min 32s, total: 9min 6s
Wall time: 6min 45s


client_id,field_0,field_1,field_2,field_3,field_4,field_5,field_6,field_7,field_8,field_9,field_10,field_11,field_12,field_13,field_14,field_15,field_16,field_17,field_18,field_19,field_20,field_21,field_22,field_23,field_24,field_25,field_26,field_27,field_28,field_29,field_30,field_31,field_32,field_33,field_34,field_35,…,field_736,field_737,field_738,field_739,field_740,field_741,field_742,field_743,field_744,field_745,field_746,field_747,field_748,field_749,field_750,field_751,field_752,field_753,field_754,field_755,field_756,field_757,field_758,field_759,field_760,field_761,field_762,field_763,field_764,field_765,field_766,field_767,new_mon,num_dials,seconds_from_last_call,seconds_from_mean_call,num_dials_lm
str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i32,u32,i64,i64,u32
"""00019eed9ed218…",0.187569,-0.11389,0.47339,-0.302068,-0.246186,0.470378,0.286578,0.262632,-0.301005,0.366354,-0.285033,-0.293067,-0.343036,-0.348693,0.420059,-0.062967,0.660817,0.220458,0.203293,-0.439991,-0.999854,-0.379996,-0.157852,-0.27415,-0.270851,0.361127,-0.317062,0.432029,0.301804,-0.296994,0.158766,-0.999393,0.822938,0.721764,0.361346,-0.154654,…,-0.52684,0.48297,-0.300226,0.235057,-0.935582,-0.324502,0.109354,0.492208,0.453366,-0.465395,-0.291755,-0.35037,-0.080593,0.1934,0.225013,0.423672,0.413629,0.321914,-0.171167,-0.434543,0.914909,-0.226921,0.385365,0.327486,0.355797,0.975489,0.438586,0.019791,0.189426,-0.311062,0.527914,0.201623,2,2,1340594,1998344,2
"""0001ac6446bf22…",0.106713,0.108001,0.325686,0.04329,-0.151876,0.297966,0.022422,0.104224,-0.044169,0.163554,0.179369,-0.030503,-0.29567,-0.17223,0.123619,-0.046759,0.466789,0.023533,-0.078267,-0.116173,-0.995523,-0.102831,-0.180166,-0.064947,-0.095537,0.140421,-0.082992,0.122801,-0.186758,0.020296,-0.121644,-0.979954,0.031154,0.423367,0.06926,0.065344,…,-0.323419,0.006433,-0.149066,0.075631,-0.605407,0.192733,-0.056283,0.164806,0.185165,-0.078068,0.194711,-0.178549,0.015922,0.158233,-0.082943,0.13822,0.004709,0.190236,-0.160643,-0.116047,0.324843,-0.117935,0.110361,0.035899,0.165119,0.248349,0.190246,0.15903,0.19767,-0.051101,0.293387,-0.144627,2,1,404312,404312,1


In [38]:
%%time
dial_res_test = []
for m in range(11,14):
    t = dial_test.filter(pol.col('event_time')<datetime.datetime(2022+(m-1)//12, m%12+int((m%12)==0)*12, 1, 0, 0, 0, 0))
    print(m, t.shape, t['client_id'].n_unique())
    t_embs = pol.from_pandas(t.to_pandas().groupby('client_id')['embedding'].mean().reset_index(drop=False))
    t_embs = t_embs.with_columns(pol.col("embedding").list.to_struct()).unnest("embedding")
    t_embs = t_embs.with_columns(pol.lit(m).alias('new_mon'))

    a = t.group_by('client_id').agg(pol.col('event_time').count().alias('num_dials'))
    t_embs = t_embs.join(a, on='client_id', how='left')
    
    a = t.group_by('client_id').agg(pol.col('event_time').max().alias('seconds_from_last_call'))
    a = a.with_columns((datetime.datetime(2022+(m-1)//12, m%12+int((m%12)==0)*12, 1, 0, 0, 0, 0)-pol.col('seconds_from_last_call')).dt.total_seconds().cast(int).alias('seconds_from_last_call'))
    t_embs = t_embs.join(a, on='client_id', how='left')    
    
    a = t.group_by('client_id').agg(pol.col('event_time').mean().alias('seconds_from_mean_call'))
    a = a.with_columns((datetime.datetime(2022+(m-1)//12, m%12+int((m%12)==0)*12, 1, 0, 0, 0, 0)-pol.col('seconds_from_mean_call')).dt.total_seconds().cast(int).alias('seconds_from_mean_call'))
    t_embs = t_embs.join(a, on='client_id', how='left')   

    t = dial_test.filter(pol.col('event_time')<datetime.datetime(2022+(m-1)//12, m%12+int((m%12)==0)*12, 1, 0, 0, 0, 0))
    t = t.filter(pol.col('event_time')>=datetime.datetime(2022+(m-2)//12, (m-1)%12+int(((m-1)%12)==0)*12, 1, 0, 0, 0, 0))

    a = t.group_by('client_id').agg(pol.col('event_time').count().alias('num_dials_lm'))
    t_embs = t_embs.join(a, on='client_id', how='left').fill_null(0)

    #t_embs1 = pol.from_pandas(t.to_pandas().groupby('client_id')['embedding'].mean().reset_index(drop=False))
    #t_embs1 = t_embs1.with_columns(pol.col("embedding").list.to_struct()).unnest("embedding")
    #t_embs1 = t_embs1.rename({'field_'+str(i):'lm_field_'+str(i) for i in range(768)})
    #t_embs = t_embs.join(t_embs1, on='client_id', how='left')
    
    dial_res_test.append(t_embs)
    del t, t_embs, a#, t_embs1

dial_res_test = pol.concat(dial_res_test)
print(dial_res_test.shape)
dial_res_test.head(2)

11 (249279, 3) 76506
12 (272735, 3) 80071
13 (283658, 3) 81425
(238002, 774)
CPU times: user 35.5 s, sys: 8.83 s, total: 44.3 s
Wall time: 31.3 s


client_id,field_0,field_1,field_2,field_3,field_4,field_5,field_6,field_7,field_8,field_9,field_10,field_11,field_12,field_13,field_14,field_15,field_16,field_17,field_18,field_19,field_20,field_21,field_22,field_23,field_24,field_25,field_26,field_27,field_28,field_29,field_30,field_31,field_32,field_33,field_34,field_35,…,field_736,field_737,field_738,field_739,field_740,field_741,field_742,field_743,field_744,field_745,field_746,field_747,field_748,field_749,field_750,field_751,field_752,field_753,field_754,field_755,field_756,field_757,field_758,field_759,field_760,field_761,field_762,field_763,field_764,field_765,field_766,field_767,new_mon,num_dials,seconds_from_last_call,seconds_from_mean_call,num_dials_lm
str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i32,u32,i64,i64,u32
"""00006c6ed6d81e…",0.092142,-0.150681,0.324699,-0.014067,-0.041489,0.244475,0.106149,0.151189,-0.148194,0.131524,-0.081849,-0.192752,-0.263233,-0.072393,0.031202,-0.167948,0.742802,0.151045,0.114667,-0.024638,-0.997663,-0.150705,-0.097386,-0.129484,-0.196692,0.178591,-0.17216,0.19534,0.224109,-0.163325,0.103932,-0.996865,0.400632,0.432596,0.192625,-0.128639,…,-0.319785,0.165124,-0.14786,0.119708,-0.997172,-0.073151,-0.000698,0.178239,0.355032,-0.195842,0.016664,-0.049518,-0.105358,0.061622,-0.120078,0.192818,0.176001,0.011319,-0.046793,-0.177614,0.666037,-0.089323,0.243261,0.097592,0.107901,0.66899,0.291956,0.0388,0.082139,-0.025007,0.194962,0.164325,11,2,10338497,15958642,0
"""0001ac6446bf22…",0.29131,-0.14879,0.514651,-0.303964,-0.251008,0.486174,0.265109,0.412243,-0.426082,0.38492,-0.302452,-0.386815,-0.478649,-0.385423,0.367057,-0.28872,0.886267,0.333008,0.330796,-0.348443,-0.998907,-0.42526,-0.416902,-0.287635,-0.387466,0.4713,-0.28244,0.348469,0.347034,-0.323238,0.266053,-0.996896,0.824529,0.819075,0.310999,-0.268026,…,-0.473746,0.418095,-0.325296,0.282971,-0.962314,-0.332863,0.234846,0.464716,0.300327,-0.394155,-0.253754,-0.344738,-0.33792,0.17264,0.252165,0.316296,0.462219,0.346682,-0.290112,-0.404267,0.921972,-0.321993,0.465941,0.333061,0.373564,0.921311,0.486129,0.253997,0.27789,-0.317567,0.498916,0.323263,11,16,4806866,7356720,0


In [40]:
del dial_train, dial_test, target_train, test_target_b

In [41]:
%%time
res = res.join(dial_res.with_columns(pol.col('new_mon').cast(pol.datatypes.Int32)), on=['client_id','new_mon'], how='left').fill_null(0)
res_valid = res_valid.join(dial_res.with_columns(pol.col('new_mon').cast(pol.datatypes.Int32)), on=['client_id','new_mon'], how='left').fill_null(0)
res_test = res_test.join(dial_res_test, on=['client_id', 'new_mon'], how='left').fill_null(0)

print(res.shape, res_valid.shape, res_test.shape)

(10169678, 1051) (768502, 1051) (140488, 1045)
CPU times: user 1min 15s, sys: 2min 25s, total: 3min 40s
Wall time: 6.03 s


In [42]:
del dial_res, dial_res_test

## 4. Geo embeddings

 - присоединяем эмбеддинги по геолокациям, посчитанные в ноутбуке geo_pyspark_v1_1_train_test

In [43]:
%%time
train_geo = pol.concat([pol.read_parquet("train_geo_pyspark_v1_1_m_1_2.parquet"), pol.read_parquet("train_geo_pyspark_v1_1_m_3_4.parquet"),
                       pol.read_parquet("train_geo_pyspark_v1_1_m_5_6.parquet"), pol.read_parquet("train_geo_pyspark_v1_1_m_7_8.parquet"),
                       pol.read_parquet("train_geo_pyspark_v1_1_m_9_10.parquet"), pol.read_parquet("train_geo_pyspark_v1_1_m_11_12.parquet")])

test_geo = pol.read_parquet("test_geo_pyspark_v1_1.parquet")

print(train_geo.shape, test_geo.shape)

(9468612, 257) (166747, 257)
CPU times: user 22.6 s, sys: 2min 50s, total: 3min 13s
Wall time: 1min 26s


In [44]:
train_geo = train_geo.rename({'emb_'+f"{i:04d}":'geo_emb_'+f"{i:04d}" for i in range(256)})
test_geo = test_geo.rename({'emb_'+f"{i:04d}":'geo_emb_'+f"{i:04d}" for i in range(256)})

In [45]:
train_geo.head(2)

client_id,geo_emb_0000,geo_emb_0001,geo_emb_0002,geo_emb_0003,geo_emb_0004,geo_emb_0005,geo_emb_0006,geo_emb_0007,geo_emb_0008,geo_emb_0009,geo_emb_0010,geo_emb_0011,geo_emb_0012,geo_emb_0013,geo_emb_0014,geo_emb_0015,geo_emb_0016,geo_emb_0017,geo_emb_0018,geo_emb_0019,geo_emb_0020,geo_emb_0021,geo_emb_0022,geo_emb_0023,geo_emb_0024,geo_emb_0025,geo_emb_0026,geo_emb_0027,geo_emb_0028,geo_emb_0029,geo_emb_0030,geo_emb_0031,geo_emb_0032,geo_emb_0033,geo_emb_0034,geo_emb_0035,…,geo_emb_0219,geo_emb_0220,geo_emb_0221,geo_emb_0222,geo_emb_0223,geo_emb_0224,geo_emb_0225,geo_emb_0226,geo_emb_0227,geo_emb_0228,geo_emb_0229,geo_emb_0230,geo_emb_0231,geo_emb_0232,geo_emb_0233,geo_emb_0234,geo_emb_0235,geo_emb_0236,geo_emb_0237,geo_emb_0238,geo_emb_0239,geo_emb_0240,geo_emb_0241,geo_emb_0242,geo_emb_0243,geo_emb_0244,geo_emb_0245,geo_emb_0246,geo_emb_0247,geo_emb_0248,geo_emb_0249,geo_emb_0250,geo_emb_0251,geo_emb_0252,geo_emb_0253,geo_emb_0254,geo_emb_0255
str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""000323fc412786…",-0.142873,0.134655,-0.002791,-0.434657,0.054647,0.395894,0.537642,-0.363013,0.435559,0.057897,-0.104825,-0.177034,0.094555,-0.124496,0.102517,0.144903,-0.39986,0.011606,0.474795,-0.225764,-0.274411,-0.174544,-0.175609,-0.390923,0.196135,-0.488576,0.426232,-0.598348,-0.441041,0.183796,-0.493474,-0.450656,0.425046,-0.473788,0.284142,0.081927,…,-0.430937,0.280916,-0.447042,0.262455,-0.041194,0.406885,-0.49511,-0.168666,-0.415749,-0.102904,-0.27304,-0.446721,-0.302773,-0.47728,0.479771,-0.226813,-0.442172,-0.359114,-0.273465,-0.08546,-0.137138,-0.42674,0.475077,0.469951,0.480556,0.257752,0.138226,-0.462892,0.373071,0.502678,0.17369,-0.095134,0.071633,0.144755,-0.365712,-0.432866,-0.495162
"""000323fc412786…",-0.199284,0.061525,0.009214,-0.51831,0.061705,0.515395,0.586071,-0.42721,0.544734,0.044781,-0.203014,-0.254162,0.15316,-0.146957,0.112048,0.55185,-0.514002,0.021557,0.584093,-0.296349,-0.448482,-0.066837,-0.108213,-0.522807,0.229233,-0.5876,0.537391,-0.555312,-0.520937,-0.013287,-0.626062,-0.551755,0.511184,-0.625334,0.26926,0.07148,…,-0.523246,0.379122,-0.575052,0.208089,0.057758,0.501613,-0.63105,-0.211633,-0.465333,-0.072892,-0.333683,-0.563626,-0.226613,-0.577883,0.569837,-0.329113,-0.548805,-0.494156,-0.275635,-0.108571,-0.156096,-0.505289,0.577751,0.558298,0.581405,0.327231,0.172167,-0.553271,0.438628,0.590471,0.287186,-0.109221,0.079413,0.231682,-0.453888,-0.529086,-0.575796


In [46]:
train_geo.head(1).to_pandas().loc[0,'client_id']

'000323fc41278634f1d46880107681d05202f220f9b31245141f13aec6d105f9_month=1'

In [47]:
%%time
train_geo = train_geo.with_columns(pol.col('client_id').map_elements(lambda x: int(x.split('=')[-1])+1).alias('new_mon'))
train_geo = train_geo.with_columns(pol.col('client_id').map_elements(lambda x: x.split('_')[0]).alias('client_id'))
train_geo.head(2)

<timed exec>:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
<timed exec>:2: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


CPU times: user 7.26 s, sys: 4.1 s, total: 11.4 s
Wall time: 11.3 s


client_id,geo_emb_0000,geo_emb_0001,geo_emb_0002,geo_emb_0003,geo_emb_0004,geo_emb_0005,geo_emb_0006,geo_emb_0007,geo_emb_0008,geo_emb_0009,geo_emb_0010,geo_emb_0011,geo_emb_0012,geo_emb_0013,geo_emb_0014,geo_emb_0015,geo_emb_0016,geo_emb_0017,geo_emb_0018,geo_emb_0019,geo_emb_0020,geo_emb_0021,geo_emb_0022,geo_emb_0023,geo_emb_0024,geo_emb_0025,geo_emb_0026,geo_emb_0027,geo_emb_0028,geo_emb_0029,geo_emb_0030,geo_emb_0031,geo_emb_0032,geo_emb_0033,geo_emb_0034,geo_emb_0035,…,geo_emb_0220,geo_emb_0221,geo_emb_0222,geo_emb_0223,geo_emb_0224,geo_emb_0225,geo_emb_0226,geo_emb_0227,geo_emb_0228,geo_emb_0229,geo_emb_0230,geo_emb_0231,geo_emb_0232,geo_emb_0233,geo_emb_0234,geo_emb_0235,geo_emb_0236,geo_emb_0237,geo_emb_0238,geo_emb_0239,geo_emb_0240,geo_emb_0241,geo_emb_0242,geo_emb_0243,geo_emb_0244,geo_emb_0245,geo_emb_0246,geo_emb_0247,geo_emb_0248,geo_emb_0249,geo_emb_0250,geo_emb_0251,geo_emb_0252,geo_emb_0253,geo_emb_0254,geo_emb_0255,new_mon
str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i64
"""000323fc412786…",-0.142873,0.134655,-0.002791,-0.434657,0.054647,0.395894,0.537642,-0.363013,0.435559,0.057897,-0.104825,-0.177034,0.094555,-0.124496,0.102517,0.144903,-0.39986,0.011606,0.474795,-0.225764,-0.274411,-0.174544,-0.175609,-0.390923,0.196135,-0.488576,0.426232,-0.598348,-0.441041,0.183796,-0.493474,-0.450656,0.425046,-0.473788,0.284142,0.081927,…,0.280916,-0.447042,0.262455,-0.041194,0.406885,-0.49511,-0.168666,-0.415749,-0.102904,-0.27304,-0.446721,-0.302773,-0.47728,0.479771,-0.226813,-0.442172,-0.359114,-0.273465,-0.08546,-0.137138,-0.42674,0.475077,0.469951,0.480556,0.257752,0.138226,-0.462892,0.373071,0.502678,0.17369,-0.095134,0.071633,0.144755,-0.365712,-0.432866,-0.495162,2
"""000323fc412786…",-0.199284,0.061525,0.009214,-0.51831,0.061705,0.515395,0.586071,-0.42721,0.544734,0.044781,-0.203014,-0.254162,0.15316,-0.146957,0.112048,0.55185,-0.514002,0.021557,0.584093,-0.296349,-0.448482,-0.066837,-0.108213,-0.522807,0.229233,-0.5876,0.537391,-0.555312,-0.520937,-0.013287,-0.626062,-0.551755,0.511184,-0.625334,0.26926,0.07148,…,0.379122,-0.575052,0.208089,0.057758,0.501613,-0.63105,-0.211633,-0.465333,-0.072892,-0.333683,-0.563626,-0.226613,-0.577883,0.569837,-0.329113,-0.548805,-0.494156,-0.275635,-0.108571,-0.156096,-0.505289,0.577751,0.558298,0.581405,0.327231,0.172167,-0.553271,0.438628,0.590471,0.287186,-0.109221,0.079413,0.231682,-0.453888,-0.529086,-0.575796,3


In [48]:
%%time
res = res.join(train_geo.with_columns(pol.col('new_mon').cast(pol.datatypes.Int32)), on=['client_id','new_mon'], how='left').fill_null(0)
res_valid = res_valid.join(train_geo.with_columns(pol.col('new_mon').cast(pol.datatypes.Int32)), on=['client_id','new_mon'], how='left').fill_null(0)
res_test = res_test.join(test_geo, on=['client_id'], how='left').fill_null(0)

print(res.shape, res_valid.shape, res_test.shape)

(10169678, 1307) (768502, 1307) (140488, 1301)
CPU times: user 55.4 s, sys: 1min 6s, total: 2min 1s
Wall time: 5.04 s


In [49]:
del train_geo, test_geo

## 5. Табличные фичи по истории транзакций

 - сколько у клиента всего было транзакций и сколько за последний месяц, сколько времени прошло с последней и средней транзакции
 - то же самое в разрезе тразакций разных типов (event_type, src_type11, dst_type11)

In [50]:
%%time
transactions_train = pol.read_parquet("Hackathon/trx_train.parquet/*")
transactions_test = pol.read_parquet("Hackathon/trx_test.parquet/*")
print(transactions_train.shape, transactions_test.shape)

(167467698, 14) (47608663, 14)
CPU times: user 36.2 s, sys: 1min 34s, total: 2min 10s
Wall time: 29.6 s


In [51]:
transactions_train.head(2)

event_time,amount,client_id,event_type,event_subtype,currency,src_type11,src_type12,dst_type11,dst_type12,src_type21,src_type22,src_type31,src_type32
datetime[μs],f32,str,i32,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64
2022-04-29 13:56:58.137690,156129.734375,"""51f88b707d9c87…",56,9,11.0,109.0,74.0,364.0,22652.0,46339.0,43.0,2423.0,25.0
2022-07-19 12:28:12.873540,357370.21875,"""51f88b707d9c87…",56,9,11.0,109.0,74.0,364.0,22652.0,46339.0,43.0,2423.0,25.0


In [52]:
transactions_test.head(2)

event_time,amount,client_id,event_type,event_subtype,currency,src_type11,src_type12,dst_type11,dst_type12,src_type21,src_type22,src_type31,src_type32
datetime[ns],f32,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
2022-09-01 22:15:48.451581,314594.15625,"""011175314a5d1c…",44,14,11,19,344,364,22652,40473,51,1738,36
2022-10-17 14:47:41.538582,5523.386719,"""011175314a5d1c…",44,14,11,19,344,364,22652,40473,51,1738,36


In [53]:
cols = transactions_test.drop(['event_time', 'amount', 'client_id']).columns
print(cols)

['event_type', 'event_subtype', 'currency', 'src_type11', 'src_type12', 'dst_type11', 'dst_type12', 'src_type21', 'src_type22', 'src_type31', 'src_type32']


Приводим данные в трейне и тесте к единому виду для объединения

In [54]:
for col in ['src_type11', 'dst_type11', 'src_type22', 'src_type32']:
    print(col, end= ' ')
    transactions_train = transactions_train.with_columns(pol.col(col).fill_null(0).cast(pol.Int32))
    transactions_test = transactions_test.with_columns(pol.col(col).fill_null(0).cast(pol.Int32))

src_type11 dst_type11 src_type22 src_type32 

In [55]:
cols = ['event_time', 'amount', 'client_id', 'event_type', 'src_type11', 'dst_type11', 'src_type22', 'src_type32']
transactions_train = pol.concat([transactions_train.with_columns(pol.col('event_time').dt.cast_time_unit("ns")).select(cols), 
                                 transactions_test.select(cols)])
print(transactions_train.shape)

(215076361, 8)


In [56]:
{c:d for c, d in zip(transactions_train.columns, transactions_train.dtypes)}

{'event_time': Datetime(time_unit='ns', time_zone=None),
 'amount': Float32,
 'client_id': String,
 'event_type': Int32,
 'src_type11': Int32,
 'dst_type11': Int32,
 'src_type22': Int32,
 'src_type32': Int32}

In [57]:
transactions_train.head(2)

event_time,amount,client_id,event_type,src_type11,dst_type11,src_type22,src_type32
datetime[ns],f32,str,i32,i32,i32,i32,i32
2022-04-29 13:56:58.137690,156129.734375,"""51f88b707d9c87…",56,109,364,43,25
2022-07-19 12:28:12.873540,357370.21875,"""51f88b707d9c87…",56,109,364,43,25


In [58]:
unique_values_dict = {}
for col in ['event_type', 'src_type11', 'dst_type11', 'src_type22', 'src_type32']:
    unique_values_dict[col] = [str(i) for i in transactions_train[col].unique().to_numpy().tolist()]
    print(col, len(unique_values_dict[col]))

event_type 56
src_type11 80
dst_type11 86
src_type22 88
src_type32 89


In [59]:
cols_to_agg = ['event_type', 'src_type11', 'dst_type11']#, 'src_type22', 'src_type32'

In [60]:
%%time
trx_res = []
for m in range(2,14):
    t = transactions_train.filter(pol.col('event_time')<datetime.datetime(2022+(m-1)//12, m%12+int((m%12)==0)*12, 1, 0, 0, 0, 0))
    print(m, t.shape, t['client_id'].n_unique())    
    
    t_embs = t.unique('client_id').select(['client_id'])
    t_embs = t_embs.with_columns(pol.lit(m).alias('new_mon'))

    a = t.group_by('client_id').agg(pol.col('event_time').count().alias('num_trxs'))
    t_embs = t_embs.join(a, on='client_id', how='left')
    
    a = t.group_by('client_id').agg(pol.col('event_time').max().alias('seconds_from_last_trx'))
    a = a.with_columns((datetime.datetime(2022+(m-1)//12, m%12+int((m%12)==0)*12, 1, 0, 0, 0, 0)-pol.col('seconds_from_last_trx')).dt.total_seconds().cast(int).alias('seconds_from_last_trx'))
    t_embs = t_embs.join(a, on='client_id', how='left')    
    
    a = t.group_by('client_id').agg(pol.col('event_time').mean().alias('seconds_from_mean_trx'))
    a = a.with_columns((datetime.datetime(2022+(m-1)//12, m%12+int((m%12)==0)*12, 1, 0, 0, 0, 0)-pol.col('seconds_from_mean_trx')).dt.total_seconds().cast(int).alias('seconds_from_mean_trx'))
    t_embs = t_embs.join(a, on='client_id', how='left')   

    d = datetime.datetime(2022+(m-1)//12, m%12+int((m%12)==0)*12, 1, 0, 0, 0, 0)

    for col in cols_to_agg:
        
        a = t.pivot(index='client_id', values='event_time', columns=col, aggregate_function='max')        
        for c in unique_values_dict[col]:
            if c not in a:
                a = a.with_columns(pol.lit(None).alias(c))
        a = a.with_columns(-(pol.col(unique_values_dict[col])-pol.lit(d)).dt.total_seconds()).fill_null(-1000000)
        a = a.rename({str(i):'seconds_from_last_'+str(i)+'_'+col for i in unique_values_dict[col]})
        t_embs = t_embs.join(a, on='client_id', how='left')
    
        a = t.pivot(index='client_id', values='event_time', columns=col, aggregate_function='mean')
        for c in unique_values_dict[col]:
            if c not in a:
                a = a.with_columns(pol.lit(None).alias(c))
        a = a.with_columns(-(pol.col(unique_values_dict[col])-pol.lit(d)).dt.total_seconds()).fill_null(-1000000)
        a = a.rename({str(i):'seconds_from_mean_'+str(i)+'_'+col for i in unique_values_dict[col]})
        t_embs = t_embs.join(a, on='client_id', how='left')
    
        a = t.pivot(index='client_id', values='event_time', columns=col, aggregate_function='len')
        for c in unique_values_dict[col]:
            if c not in a:
                a = a.with_columns(pol.lit(None).alias(c))
        a = a.fill_null(-100)
        a = a.rename({str(i):'num_'+str(i)+'_'+col for i in unique_values_dict[col]})
        t_embs = t_embs.join(a, on='client_id', how='left')

    t = transactions_train.filter(pol.col('event_time')<datetime.datetime(2022+(m-1)//12, m%12+int((m%12)==0)*12, 1, 0, 0, 0, 0))
    t = t.filter(pol.col('event_time')>=datetime.datetime(2022+(m-2)//12, (m-1)%12+int(((m-1)%12)==0)*12, 1, 0, 0, 0, 0))

    a = t.group_by('client_id').agg(pol.col('event_time').count().alias('num_trxs_lm')).fill_null(0)
    t_embs = t_embs.join(a, on='client_id', how='left')

    for col in cols_to_agg:
        a = t.pivot(index='client_id', values='event_time', columns=col, aggregate_function='len')
        for c in unique_values_dict[col]:
            if c not in a:
                a = a.with_columns(pol.lit(None).alias(c))
        a = a.fill_null(-100)
        a = a.rename({str(i):'lm_num_'+str(i)+'_'+col for i in unique_values_dict[col]})
        t_embs = t_embs.join(a, on='client_id', how='left')

        for c in ['seconds_from_last_'+str(i)+'_'+col for i in unique_values_dict[col]]:
            t_embs = t_embs.with_columns(pol.col(c).fill_null(-1000000).cast(pol.Int32))
        
        for c in ['seconds_from_mean_'+str(i)+'_'+col for i in unique_values_dict[col]]:
            t_embs = t_embs.with_columns(pol.col(c).fill_null(-1000000).cast(pol.Int32))
        
        for c in ['num_'+str(i)+'_'+col for i in unique_values_dict[col]]:
            t_embs = t_embs.with_columns(pol.col(c).fill_null(-100).cast(pol.Int32))
        
        for c in ['lm_num_'+str(i)+'_'+col for i in unique_values_dict[col]]:
            t_embs = t_embs.with_columns(pol.col(c).fill_null(-100).cast(pol.Int32))
    
    trx_res.append(t_embs)
    del t, t_embs, a

cols = trx_res[0].columns
trx_res = pol.concat([i.select(cols) for i in trx_res])
print(trx_res.shape)
trx_res.head(2)

2 (17183111, 8) 847334


Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:40:44 UTC)" was missed by 0:00:02.351162


3 (34648795, 8) 899797


Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:41:24 UTC)" was missed by 0:00:02.234134


4 (54798981, 8) 934420


...

Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:50:24 UTC)" was missed by 0:00:02.347922
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:50:34 UTC)" was missed by 0:00:01.215344
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:50:54 UTC)" was missed by 0:00:01.103218


13 (215076361, 8) 1081371


Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:51:44 UTC)" was missed by 0:00:03.406454
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:51:54 UTC)" was missed by 0:00:01.671905
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:52:04 UTC)" was missed by 0:00:01.326578
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:52:54 UTC)" was missed by 0:00:05.350787


(11966189, 894)
CPU times: user 2h 6min 8s, sys: 48min 44s, total: 2h 54min 53s
Wall time: 12min 56s


client_id,new_mon,num_trxs,seconds_from_last_trx,seconds_from_mean_trx,seconds_from_last_40_event_type,seconds_from_last_35_event_type,seconds_from_last_52_event_type,seconds_from_last_5_event_type,seconds_from_last_54_event_type,seconds_from_last_56_event_type,seconds_from_last_38_event_type,seconds_from_last_4_event_type,seconds_from_last_25_event_type,seconds_from_last_7_event_type,seconds_from_last_51_event_type,seconds_from_last_37_event_type,seconds_from_last_12_event_type,seconds_from_last_45_event_type,seconds_from_last_53_event_type,seconds_from_last_48_event_type,seconds_from_last_16_event_type,seconds_from_last_27_event_type,seconds_from_last_44_event_type,seconds_from_last_15_event_type,seconds_from_last_41_event_type,seconds_from_last_20_event_type,seconds_from_last_30_event_type,seconds_from_last_31_event_type,seconds_from_last_42_event_type,seconds_from_last_9_event_type,seconds_from_last_50_event_type,seconds_from_last_10_event_type,seconds_from_last_47_event_type,seconds_from_last_8_event_type,seconds_from_last_49_event_type,seconds_from_last_23_event_type,…,lm_num_387_dst_type11,lm_num_1345_dst_type11,lm_num_1141_dst_type11,lm_num_1703_dst_type11,lm_num_1390_dst_type11,lm_num_827_dst_type11,lm_num_1335_dst_type11,lm_num_819_dst_type11,lm_num_800_dst_type11,lm_num_1027_dst_type11,lm_num_310_dst_type11,lm_num_1653_dst_type11,lm_num_1661_dst_type11,lm_num_60_dst_type11,lm_num_752_dst_type11,lm_num_263_dst_type11,lm_num_1440_dst_type11,lm_num_1655_dst_type11,lm_num_55_dst_type11,lm_num_107_dst_type11,lm_num_144_dst_type11,lm_num_156_dst_type11,lm_num_219_dst_type11,lm_num_356_dst_type11,lm_num_537_dst_type11,lm_num_617_dst_type11,lm_num_793_dst_type11,lm_num_856_dst_type11,lm_num_881_dst_type11,lm_num_970_dst_type11,lm_num_1077_dst_type11,lm_num_1190_dst_type11,lm_num_1242_dst_type11,lm_num_1362_dst_type11,lm_num_1399_dst_type11,lm_num_1456_dst_type11,lm_num_1622_dst_type11
str,i32,u32,i64,i64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""3b1c34136e3b46…",2,13,307378,1539699,-1000000,-1000000,-1000000,-1000000,307378,-1000000,498674,-1000000,-1000000,-1000000,-1000000,1179147,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,…,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
"""2285fbd5c99bed…",2,1,1907278,1907278,-1000000,-1000000,-1000000,1907278,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,…,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100


In [61]:
%%time
trx_res_test = []
for m in range(11,14):
    t = transactions_test.filter(pol.col('event_time')<datetime.datetime(2022+(m-1)//12, m%12+int((m%12)==0)*12, 1, 0, 0, 0, 0))
    print(m, t.shape, t['client_id'].n_unique())    
    
    t_embs = t.unique('client_id').select(['client_id'])
    t_embs = t_embs.with_columns(pol.lit(m).alias('new_mon'))

    a = t.group_by('client_id').agg(pol.col('event_time').count().alias('num_trxs'))
    t_embs = t_embs.join(a, on='client_id', how='left')
    
    a = t.group_by('client_id').agg(pol.col('event_time').max().alias('seconds_from_last_trx'))
    a = a.with_columns((datetime.datetime(2022+(m-1)//12, m%12+int((m%12)==0)*12, 1, 0, 0, 0, 0)-pol.col('seconds_from_last_trx')).dt.total_seconds().cast(int).alias('seconds_from_last_trx'))
    t_embs = t_embs.join(a, on='client_id', how='left')    
    
    a = t.group_by('client_id').agg(pol.col('event_time').mean().alias('seconds_from_mean_trx'))
    a = a.with_columns((datetime.datetime(2022+(m-1)//12, m%12+int((m%12)==0)*12, 1, 0, 0, 0, 0)-pol.col('seconds_from_mean_trx')).dt.total_seconds().cast(int).alias('seconds_from_mean_trx'))
    t_embs = t_embs.join(a, on='client_id', how='left')   

    d = datetime.datetime(2022+(m-1)//12, m%12+int((m%12)==0)*12, 1, 0, 0, 0, 0)

    for col in cols_to_agg:
        
        a = t.pivot(index='client_id', values='event_time', columns=col, aggregate_function='max')
        for c in unique_values_dict[col]:
            if c not in a:
                a = a.with_columns(pol.lit(None).alias(c))
        a = a.with_columns(-(pol.col(unique_values_dict[col])-pol.lit(d)).dt.total_seconds()).fill_null(-1000000)
        a = a.rename({str(i):'seconds_from_last_'+str(i)+'_'+col for i in unique_values_dict[col]})
        t_embs = t_embs.join(a, on='client_id', how='left')
    
        a = t.pivot(index='client_id', values='event_time', columns=col, aggregate_function='mean')
        for c in unique_values_dict[col]:
            if c not in a:
                a = a.with_columns(pol.lit(None).alias(c))
        a = a.with_columns(-(pol.col(unique_values_dict[col])-pol.lit(d)).dt.total_seconds()).fill_null(-1000000)
        a = a.rename({str(i):'seconds_from_mean_'+str(i)+'_'+col for i in unique_values_dict[col]})
        t_embs = t_embs.join(a, on='client_id', how='left')
    
        a = t.pivot(index='client_id', values='event_time', columns=col, aggregate_function='len')
        for c in unique_values_dict[col]:
            if c not in a:
                a = a.with_columns(pol.lit(None).alias(c))
        a = a.fill_null(-100)
        a = a.rename({str(i):'num_'+str(i)+'_'+col for i in unique_values_dict[col]})
        t_embs = t_embs.join(a, on='client_id', how='left')

    t = transactions_test.filter(pol.col('event_time')<datetime.datetime(2022+(m-1)//12, m%12+int((m%12)==0)*12, 1, 0, 0, 0, 0))
    t = t.filter(pol.col('event_time')>=datetime.datetime(2022+(m-2)//12, (m-1)%12+int(((m-1)%12)==0)*12, 1, 0, 0, 0, 0))

    a = t.group_by('client_id').agg(pol.col('event_time').count().alias('num_trxs_lm')).fill_null(0)
    t_embs = t_embs.join(a, on='client_id', how='left')

    for col in cols_to_agg:
        a = t.pivot(index='client_id', values='event_time', columns=col, aggregate_function='len')
        for c in unique_values_dict[col]:
            if c not in a:
                a = a.with_columns(pol.lit(None).alias(c))
        a = a.fill_null(-100)
        a = a.rename({str(i):'lm_num_'+str(i)+'_'+col for i in unique_values_dict[col]})
        t_embs = t_embs.join(a, on='client_id', how='left')

        for c in ['seconds_from_last_'+str(i)+'_'+col for i in unique_values_dict[col]]:
            t_embs = t_embs.with_columns(pol.col(c).fill_null(-1000000).cast(pol.Int32))
        
        for c in ['seconds_from_mean_'+str(i)+'_'+col for i in unique_values_dict[col]]:
            t_embs = t_embs.with_columns(pol.col(c).fill_null(-1000000).cast(pol.Int32))
        
        for c in ['num_'+str(i)+'_'+col for i in unique_values_dict[col]]:
            t_embs = t_embs.with_columns(pol.col(c).fill_null(-100).cast(pol.Int32))
        
        for c in ['lm_num_'+str(i)+'_'+col for i in unique_values_dict[col]]:
            t_embs = t_embs.with_columns(pol.col(c).fill_null(-100).cast(pol.Int32))

    trx_res_test.append(t_embs)
    del t, t_embs, a

cols = trx_res_test[0].columns
trx_res_test = pol.concat([i.select(cols) for i in trx_res_test])
print(trx_res_test.shape)
trx_res_test.head(2)

11 (44566857, 14) 224074


Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:53:24 UTC)" was missed by 0:00:01.024932


12 (47299989, 14) 225633
13 (47608663, 14) 225744


Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:53:54 UTC)" was missed by 0:00:01.325144


(675451, 894)
CPU times: user 9min 45s, sys: 3min 26s, total: 13min 11s
Wall time: 59.4 s


client_id,new_mon,num_trxs,seconds_from_last_trx,seconds_from_mean_trx,seconds_from_last_44_event_type,seconds_from_last_25_event_type,seconds_from_last_15_event_type,seconds_from_last_54_event_type,seconds_from_last_40_event_type,seconds_from_last_52_event_type,seconds_from_last_37_event_type,seconds_from_last_5_event_type,seconds_from_last_51_event_type,seconds_from_last_38_event_type,seconds_from_last_41_event_type,seconds_from_last_50_event_type,seconds_from_last_45_event_type,seconds_from_last_23_event_type,seconds_from_last_48_event_type,seconds_from_last_47_event_type,seconds_from_last_14_event_type,seconds_from_last_56_event_type,seconds_from_last_4_event_type,seconds_from_last_10_event_type,seconds_from_last_8_event_type,seconds_from_last_53_event_type,seconds_from_last_9_event_type,seconds_from_last_7_event_type,seconds_from_last_16_event_type,seconds_from_last_24_event_type,seconds_from_last_32_event_type,seconds_from_last_20_event_type,seconds_from_last_35_event_type,seconds_from_last_12_event_type,seconds_from_last_42_event_type,seconds_from_last_19_event_type,…,lm_num_793_dst_type11,lm_num_1047_dst_type11,lm_num_1242_dst_type11,lm_num_827_dst_type11,lm_num_1027_dst_type11,lm_num_1661_dst_type11,lm_num_1110_dst_type11,lm_num_55_dst_type11,lm_num_60_dst_type11,lm_num_107_dst_type11,lm_num_144_dst_type11,lm_num_156_dst_type11,lm_num_201_dst_type11,lm_num_219_dst_type11,lm_num_310_dst_type11,lm_num_356_dst_type11,lm_num_387_dst_type11,lm_num_537_dst_type11,lm_num_617_dst_type11,lm_num_752_dst_type11,lm_num_819_dst_type11,lm_num_856_dst_type11,lm_num_881_dst_type11,lm_num_938_dst_type11,lm_num_970_dst_type11,lm_num_1077_dst_type11,lm_num_1190_dst_type11,lm_num_1335_dst_type11,lm_num_1345_dst_type11,lm_num_1362_dst_type11,lm_num_1390_dst_type11,lm_num_1399_dst_type11,lm_num_1434_dst_type11,lm_num_1440_dst_type11,lm_num_1456_dst_type11,lm_num_1622_dst_type11,lm_num_1655_dst_type11
str,i32,u32,i64,i64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""fc9b2a0c781276…",11,171,336008,11527451,2366101,-1000000,-1000000,4380231,877039,2628155,336008,4337488,-1000000,-1000000,-1000000,1596438,-1000000,-1000000,4455048,-1000000,-1000000,561726,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,1094553,-1000000,638394,-1000000,-1000000,-1000000,-1000000,-1000000,…,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
"""40e1024a3cfa5a…",11,315,34275,12427305,-1000000,39699,-1000000,34275,2341956,-1000000,1744073,-1000000,-1000000,-1000000,2370550,-1000000,24053782,-1000000,-1000000,-1000000,-1000000,24668884,5213328,-1000000,-1000000,-1000000,-1000000,13527153,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,…,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100


In [62]:
del transactions_train, transactions_test

In [63]:
%%time
print(res.shape, res_valid.shape, res_test.shape)

res = res.join(trx_res.with_columns(pol.col('new_mon').cast(pol.datatypes.Int32)), on=['client_id','new_mon'], how='left').fill_null(0)
res_valid = res_valid.join(trx_res.with_columns(pol.col('new_mon').cast(pol.datatypes.Int32)), on=['client_id','new_mon'], how='left').fill_null(0)
res_test = res_test.join(trx_res_test.with_columns(pol.col('new_mon').cast(pol.datatypes.Int32)), on=['client_id', 'new_mon'], how='left').fill_null(0)

print(res.shape, res_valid.shape, res_test.shape)

(10169678, 1307) (768502, 1307) (140488, 1301)
(10169678, 2199) (768502, 2199) (140488, 2193)
CPU times: user 5min 49s, sys: 2min 25s, total: 8min 14s
Wall time: 13.2 s


In [64]:
del trx_res, trx_res_test

## 6. Тренировка и инференс модели

Catboost не работает с датафреймами polars, поэтому переведём данные в датафреймы pandas

In [65]:
res_p = res.to_pandas()
res_valid_p = res_valid.to_pandas()
res_test_p = res_test.to_pandas()

In [66]:
del res, res_valid, res_test

In [67]:
pred_cols = ['target_1_cumsum', 'target_2_cumsum', 'target_3_cumsum', 'target_4_cumsum', 'target_cumsum',
             'last_target_1_month', 'last_target_2_month', 'last_target_3_month', 'last_target_4_month', 'last_target_month',
             'delta_last_target_1_month', 'delta_last_target_2_month', 'delta_last_target_3_month', 'delta_last_target_4_month', 
             'delta_last_target_month'] 
pred_cols = pred_cols + ['num_dials', 'seconds_from_last_call', 'seconds_from_mean_call', 'num_dials_lm']
pred_cols = pred_cols + ['num_trxs', 'seconds_from_last_trx', 'seconds_from_mean_trx', 'num_trxs_lm']
pred_cols = pred_cols + ['emb_'+f"{i:04d}" for i in range(256)] + ['field_'+str(i) for i in range(768)] + ['geo_emb_'+f"{i:04d}" for i in range(256)]

#pred_cols = pred_cols + ['lm_emb_'+f"{i:04d}" for i in range(256)]

for col in cols_to_agg:
    pred_cols = pred_cols + ['seconds_from_last_'+str(i)+'_'+col for i in unique_values_dict[col]]
    pred_cols = pred_cols + ['seconds_from_mean_'+str(i)+'_'+col for i in unique_values_dict[col]]
    pred_cols = pred_cols + ['num_'+str(i)+'_'+col for i in unique_values_dict[col]]
    pred_cols = pred_cols + ['lm_num_'+str(i)+'_'+col for i in unique_values_dict[col]]

In [68]:
RANDOM_STATE = 37

In [69]:
res_p = pd.concat([res_p, res_valid_p], axis=0)
del res_valid_p

Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:54:44 UTC)" was missed by 0:00:06.980458
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:55:04 UTC)" was missed by 0:00:01.276666
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:55:14 UTC)" was missed by 0:00:01.478817
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:55:24 UTC)" was missed by 0:00:07.768805
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:55:44 UTC)" was missed by 0:00:01.892849


In [70]:
res_p.reset_index(drop=True, inplace=True)

In [72]:
n_folds = 5
skf = StratifiedGroupKFold(n_splits=n_folds, shuffle=True, random_state=RANDOM_STATE)
#res_pred = res_p[['client_id', 'new_mon', 'target_1', 'target_2', 'target_3', 'target_4']].copy(deep=True)

res_p['pred_1'] = 0.0
res_p['pred_2'] = 0.0
res_p['pred_3'] = 0.0
res_p['pred_4'] = 0.0

#res_valid_p['pred_1'] = 0.0
#res_valid_p['pred_2'] = 0.0
#res_valid_p['pred_3'] = 0.0
#res_valid_p['pred_4'] = 0.0

res_test_p['target_1'] = 0.0
res_test_p['target_2'] = 0.0
res_test_p['target_3'] = 0.0
res_test_p['target_4'] = 0.0

#res_pred['fold'] = -1
models = []
models_send = []
scores_all = []

for n in range(1,5):
    print('='*80)
    tar = 'target_'+str(n)
    pr = 'pred_'+str(n)
    print(tar)
    print('='*30)
    scores = []
    for i, (train_index, valid_index) in enumerate(skf.split(res_p, res_p[tar], res_p['client_id'])):
        print('='*80)
        print('Fold', i)

        train_dataset = cat.Pool(res_p.loc[train_index, pred_cols], res_p.loc[train_index, tar]) 
        eval_dataset = cat.Pool(res_p.loc[valid_index, pred_cols], res_p.loc[valid_index, tar]) 
    
        model = cat.CatBoostClassifier(iterations=600,
                                  learning_rate=0.1,
                                  depth=6,
                               eval_metric='AUC',#'Logloss',
                               #task_type="GPU",
                               #devices='0:1',
                                      random_seed=RANDOM_STATE)
        # Fit model
        model.fit(train_dataset, eval_set=eval_dataset)
        # Get predictions
        preds = model.predict_proba(res_p.loc[valid_index, pred_cols])[:,1]
        res_p.loc[valid_index, pr] = preds
        models.append(model)

        del train_dataset, eval_dataset, preds

        res_test_p[tar] = res_test_p[tar].values + model.predict_proba(res_test_p.loc[:, pred_cols])[:,1]/n_folds
    
        del model#, p#, b, c

    
    print('='*80)
    #print(tar, np.mean(scores), '|', scores)
    print(tar, 'cv total scores, oof prediction', roc_auc_score(res_p.loc[:, tar], res_p.loc[:, pr]))


    # full train run

    train_dataset = cat.Pool(res_p.loc[:, pred_cols], res_p.loc[:, tar]) 
    model = cat.CatBoostClassifier(iterations=600,
                                  learning_rate=0.1,
                                  depth=6,
                               eval_metric='AUC',#'Logloss',
                               #task_type="GPU",
                               #devices='0:1',
                                      random_seed=RANDOM_STATE)
    model.fit(train_dataset)
    models.append(model)
    del train_dataset

    res_test_p[tar] = res_test_p[tar].values*5/6 + model.predict_proba(res_test_p.loc[:, pred_cols])[:,1]*1/6

    res_test_p[['client_id', 'target_1', 'target_2', 'target_3', 'target_4']].to_csv('submission_naive_v1_trx_v1_2_dial_geo_v1_1_new_fea111_plusfulltrainvalid_iter600.csv')

    with open('models_naive_v1_trx_v1_2_dial_geo_v1_1_new_fea111_plusfulltrainvalid_iter600.pickle', 'wb') as f:
        pickle.dump(models, f)

print('='*50)
#print(np.mean(scores_all), '|', scores_all)

target_1


Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 22:20:54 UTC)" was missed by 0:00:01.488982
Execution of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 22:21:44 UTC)" skipped: maximum number of running instances reached (10)
Execution of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 22:22:14 UTC)" skipped: maximum number of running instances reached (10)
Execution of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 22:22:44 UTC)" skipped: maximum number of running instances reached (10)
Execution of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 22:23:04 UTC)" skipped: maximum number of running instances reached (10)
Execution of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 22:24:24 UTC)" skipped: maximum number of running instances reached (10)
Run time of job "Track

Fold 0


Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 22:27:04 UTC)" was missed by 0:00:06.993980
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 22:28:14 UTC)" was missed by 0:00:03.476790
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 22:29:14 UTC)" was missed by 0:00:06.008245
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 22:29:34 UTC)" was missed by 0:00:11.741152
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 22:29:34 UTC)" was missed by 0:00:01.742788
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 22:29:44 UTC)" was missed by 0:00:08.259125
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 22:30:14 UTC)" was missed by 0:00:09.866504
Run time of job "Tracker._func_for

0:	test: 0.6211010	best: 0.6211010 (0)	total: 5.7s	remaining: 56m 55s
1:	test: 0.6233662	best: 0.6233662 (1)	total: 7.05s	remaining: 35m 8s
2:	test: 0.6280018	best: 0.6280018 (2)	total: 8.44s	remaining: 27m 59s
3:	test: 0.7107856	best: 0.7107856 (3)	total: 10.1s	remaining: 24m 58s
4:	test: 0.7359017	best: 0.7359017 (4)	total: 17.9s	remaining: 35m 31s
5:	test: 0.7463933	best: 0.7463933 (5)	total: 20s	remaining: 33m 3s
6:	test: 0.7692975	best: 0.7692975 (6)	total: 25.9s	remaining: 36m 30s
7:	test: 0.7717642	best: 0.7717642 (7)	total: 27.4s	remaining: 33m 47s
8:	test: 0.7749853	best: 0.7749853 (8)	total: 29.2s	remaining: 31m 59s
9:	test: 0.8020113	best: 0.8020113 (9)	total: 36.5s	remaining: 35m 56s
10:	test: 0.8081838	best: 0.8081838 (10)	total: 38.7s	remaining: 34m 30s
11:	test: 0.8088166	best: 0.8088166 (11)	total: 40.4s	remaining: 32m 58s
12:	test: 0.8135387	best: 0.8135387 (12)	total: 46.2s	remaining: 34m 47s
13:	test: 0.8181241	best: 0.8181241 (13)	total: 48.3s	remaining: 33m 43s
14:

...

Execution of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-16 13:02:14 UTC)" skipped: maximum number of running instances reached (10)


589:	total: 36m 12s	remaining: 36.8s
590:	total: 36m 18s	remaining: 33.2s
591:	total: 36m 19s	remaining: 29.5s
592:	total: 36m 21s	remaining: 25.8s
593:	total: 36m 29s	remaining: 22.1s
594:	total: 36m 31s	remaining: 18.4s
595:	total: 36m 39s	remaining: 14.8s
596:	total: 36m 41s	remaining: 11.1s


Execution of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-16 13:02:44 UTC)" skipped: maximum number of running instances reached (10)


597:	total: 36m 43s	remaining: 7.37s
598:	total: 36m 46s	remaining: 3.68s
599:	total: 36m 49s	remaining: 0us


Execution of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-16 13:03:04 UTC)" skipped: maximum number of running instances reached (10)


In [73]:
res_test_p[['client_id', 'target_1', 'target_2', 'target_3', 'target_4']].to_csv('submission_naive_v1_trx_v1_2_dial_geo_v1_1_new_fea111_plusfulltrainvalid_iter600.csv')

In [74]:
with open('models_naive_v1_trx_v1_2_dial_geo_v1_1_new_fea111_plusfulltrainvalid_iter600.pickle', 'wb') as f:
    pickle.dump(models, f)

Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-16 13:03:24 UTC)" was missed by 0:00:02.028375


In [75]:
fi = pd.DataFrame({'col':pred_cols})
for i, model in enumerate(models):
    #if i not in [5, 11, 17, 23]:
    fi[str(i)] = model.feature_importances_
fi['avg'] = fi[[str(i) for i in range(len(models)) if i not in ['5','11','17','23']]].mean(axis=1)
fi.sort_values('avg')

,col,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,avg
1643,seconds_from_mean_90_src_type11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1625,seconds_from_mean_34_src_type11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1626,seconds_from_mean_36_src_type11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1628,seconds_from_mean_44_src_type11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1631,seconds_from_mean_57_src_type11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,emb_0227,11.786733,3.224204,0.004616,0.022667,0.978895,0.435450,0.000000,2.213145,0.000000,...,0.000428,0.012901,0.001915,2.117868,6.823568,0.123084,4.279485,0.015887,17.235427,2.249057
94,emb_0071,1.504378,3.146116,26.274621,0.533380,0.765198,0.756771,0.007190,0.384850,0.431072,...,1.250535,0.000758,0.117460,2.583022,3.594463,0.104638,0.080922,4.256225,0.773904,2.289947
116,emb_0093,1.825563,0.002710,1.458817,6.824278,14.811913,0.819882,14.195179,0.517109,5.417429,...,0.509573,0.000000,0.002769,3.841834,0.006403,0.009839,2.641542,0.019043,0.000000,2.425944
270,emb_0247,6.540233,10.888138,0.000000,0.459218,0.026082,7.958418,0.000000,0.215073,0.000000,...,7.464166,3.249334,0.000000,3.538163,2.612990,1.144695,0.000000,4.564147,3.382583,2.442915


In [76]:
fi.to_csv('feature_importances_naive_v1_trx_v1_2_dial_geo_v1_1_new_fea111_plusfulltrainvalid_iter600.csv')

In [77]:
fi.set_index('col', inplace=True)

In [78]:
fi.loc['mean_emb', :] = fi.loc[['emb_'+f"{i:04d}" for i in range(256)]].mean()
fi.loc['mean_dial_emb', :] = fi.loc[['field_'+str(i) for i in range(768)]].mean()
fi.loc['mean_geo_emb', :] = fi.loc[['geo_emb_'+f"{i:04d}" for i in range(256)]].mean()
#fi.loc['mean_lm_emb', :] = fi.loc[['lm_emb_'+f"{i:04d}" for i in range(256)]].mean()

fi.loc['sum_emb', :] = fi.loc[['emb_'+f"{i:04d}" for i in range(256)]].sum()
fi.loc['sum_dial_emb', :] = fi.loc[['field_'+str(i) for i in range(768)]].sum()
fi.loc['sum_geo_emb', :] = fi.loc[['geo_emb_'+f"{i:04d}" for i in range(256)]].sum()
#fi.loc['sum_lm_emb', :] = fi.loc[['lm_emb_'+f"{i:04d}" for i in range(256)]].sum()

fi.drop(['emb_'+f"{i:04d}" for i in range(256)], axis=0, inplace=True)
fi.drop(['field_'+str(i) for i in range(768)], axis=0, inplace=True)
fi.drop(['geo_emb_'+f"{i:04d}" for i in range(256)], axis=0, inplace=True)
#fi.drop(['lm_emb_'+f"{i:04d}" for i in range(256)], axis=0, inplace=True)

In [79]:
fi.sort_values('avg')

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,avg
col,,,,,,,,,,,,,,,,,,,,,
seconds_from_mean_63_src_type11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
seconds_from_mean_1686_dst_type11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
lm_num_28_event_type,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
seconds_from_mean_537_dst_type11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
seconds_from_mean_793_dst_type11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
lm_num_19_src_type11,0.515057,0.654391,0.576672,0.373624,1.487696,0.642973,3.856150,3.928924,4.823276,3.600301,...,0.033573,0.523270,0.011649,2.120625,1.801920,2.837627,1.614285,2.039823,1.946596,1.821113
sum_dial_emb,2.534814,2.627880,1.650399,2.919264,2.749917,2.754423,2.696616,2.385843,2.530840,2.691295,...,2.172319,2.392575,3.202152,2.050831,2.140667,3.098521,1.912487,2.699338,1.898750,2.550363
sum_geo_emb,2.314571,2.280478,1.831815,2.793467,2.948607,3.076169,3.334888,3.547354,5.335294,5.984323,...,1.564996,1.819751,2.837320,1.903274,2.771060,2.941386,1.765826,2.458015,1.558638,2.935046


In [80]:
for col in cols_to_agg:
    fi.loc['mean_seconds_from_last_'+col, :] = fi.loc[['seconds_from_last_'+str(i)+'_'+col for i in unique_values_dict[col]]].mean()
    fi.loc['mean_seconds_from_mean_'+col, :] = fi.loc[['seconds_from_mean_'+str(i)+'_'+col for i in unique_values_dict[col]]].mean()
    fi.loc['mean_num_'+col, :] = fi.loc[['num_'+str(i)+'_'+col for i in unique_values_dict[col]]].mean()
    fi.loc['mean_lm_num_'+col, :] = fi.loc[['lm_num_'+str(i)+'_'+col for i in unique_values_dict[col]]].mean()
    
    fi.loc['sum_seconds_from_last_'+col, :] = fi.loc[['seconds_from_last_'+str(i)+'_'+col for i in unique_values_dict[col]]].sum()
    fi.loc['sum_seconds_from_mean_'+col, :] = fi.loc[['seconds_from_mean_'+str(i)+'_'+col for i in unique_values_dict[col]]].sum()
    fi.loc['sum_num_'+col, :] = fi.loc[['num_'+str(i)+'_'+col for i in unique_values_dict[col]]].sum()
    fi.loc['sum_lm_num_'+col, :] = fi.loc[['lm_num_'+str(i)+'_'+col for i in unique_values_dict[col]]].sum()
    
    fi.drop(['seconds_from_last_'+str(i)+'_'+col for i in unique_values_dict[col]], axis=0, inplace=True)
    fi.drop(['seconds_from_mean_'+str(i)+'_'+col for i in unique_values_dict[col]], axis=0, inplace=True)
    fi.drop(['num_'+str(i)+'_'+col for i in unique_values_dict[col]], axis=0, inplace=True)
    fi.drop(['lm_num_'+str(i)+'_'+col for i in unique_values_dict[col]], axis=0, inplace=True)

In [81]:
fi.sort_values('avg')

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,avg
col,,,,,,,,,,,,,,,,,,,,,
num_dials,0.000580,0.007011,0.000442,0.009021,0.008021,0.005483,0.002578,0.000506,0.001286,0.000183,...,0.000367,0.000661,0.000000,0.000961,0.000090,0.004097,0.000711,0.005124,0.003422,0.002186
mean_dial_emb,0.003301,0.003422,0.002149,0.003801,0.003581,0.003586,0.003511,0.003107,0.003295,0.003504,...,0.002829,0.003115,0.004169,0.002670,0.002787,0.004035,0.002490,0.003515,0.002472,0.003321
mean_geo_emb,0.009041,0.008908,0.007156,0.010912,0.011518,0.012016,0.013027,0.013857,0.020841,0.023376,...,0.006113,0.007108,0.011083,0.007435,0.010824,0.011490,0.006898,0.009602,0.006088,0.011465
mean_seconds_from_last_event_type,0.019764,0.005058,0.015146,0.008542,0.004338,0.010049,0.028877,0.022845,0.038846,0.028313,...,0.012496,0.004827,0.006652,0.009844,0.006441,0.019974,0.007600,0.007011,0.008520,0.016956
mean_seconds_from_mean_dst_type11,0.006967,0.032484,0.014321,0.018751,0.027081,0.014584,0.095769,0.014517,0.017076,0.013066,...,0.010939,0.020529,0.019023,0.002703,0.005161,0.031768,0.003701,0.005913,0.004282,0.024621
mean_num_event_type,0.007988,0.014134,0.006224,0.012323,0.007603,0.017616,0.008601,0.019303,0.020427,0.011982,...,0.008168,0.059265,0.084770,0.018556,0.017601,0.140110,0.015409,0.015992,0.015646,0.025201
last_target_3_month,0.014125,0.000000,0.002485,0.008784,0.002317,0.002296,0.010741,0.000000,0.001988,0.001136,...,0.064856,0.104358,0.139921,0.000932,0.000641,0.000845,0.003913,0.001913,0.002097,0.025309
delta_last_target_1_month,0.096371,0.093267,0.100050,0.105299,0.134914,0.115403,0.000000,0.000094,0.000277,0.000000,...,0.004898,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027369
mean_seconds_from_mean_event_type,0.023577,0.010635,0.010034,0.027351,0.017382,0.049957,0.047650,0.039218,0.024170,0.060565,...,0.021417,0.032573,0.056320,0.007373,0.010134,0.009433,0.004926,0.021145,0.009018,0.029380


In [82]:
tracker.stop()